# 실험일지 01-2
- 모델적용
    - Logistic Regression
- max_features = 140000 / grid : 0.568353(rank:1) / kaggle : 0.52893
- max_features = 120000 / grid : 0.567046(rank:5) / kaggle : 0.52674
- max_features = 100000 / grid : 0.567885(rank:3) / kaggle : 0.52899
- skfold = 5, mf 120000이 1등 
- 이를 토대로 skfold = 10으로 유지

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [52]:
def kaggle_format(df):
    df['label'][df['label'] == 'none'] = 0
    df['label'][df['label'] == 'offensive'] = 1
    df['label'][df['label'] == 'hate'] = 2
    return df

### 1. 데이터 읽기

In [4]:
df = pd.read_csv('total_20210121.csv')
df = df[['title', 'comments', 'hate']]
df.columns = ['title', 'comments', 'label']
# df['label'][df['label'] == 'none'] = 0 
# df['label'][df['label'] == 'offensive'] = 1
# df['label'][df['label'] == 'hate'] = 2
df.head()

,title,comments,label
0,"휘성, 오늘 부친상 비보..28일 발인",한남이지만 명복은 빌어드릴께요....,offensive
1,"휘성, 오늘 부친상 비보..28일 발인",울지마 바보야...나 정말 갠차나,none
2,"휘성, 오늘 부친상 비보..28일 발인",결혼까지 생각했어 같은집 같은방에서 슬퍼도 bye bye bye~~~~#,offensive
3,"휘성, 에이미 '프로포폴 공범' 지목…케이윌까지 '대략난감'[종합]",에이미 물귀신 작전이 휘(발)성 이벤트면 좋겠는데...,offensive
4,"휘성, 에이미 '프로포폴 공범' 지목…케이윌까지 '대략난감'[종합]",아무튼 여자조심하자한순간에 인생쫑난다,hate


## 2. Logistic Regression 모델 적용
- tfidf, lgs - GridSearchCV
- 데이터가 독립적이고 동일한 분포를 가진 경우
    - KFold, RepeatedKFold, LeaveOneOut(LOO), LeavePOutLeaveOneOut(LPO)
- 동일한 분포가 아닌 경우
    - StratifiedKFold, RepeatedStratifiedKFold, StratifiedShuffleSplit

In [63]:
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [67]:
kf = KFold(n_splits=10)
skfold = StratifiedKFold(n_splits=10)

In [68]:
mf_grid = {
    'tfidf__max_features': [15000, 100000, 110000, 120000, 130000, 140000, 150000],
}
pipeline = Pipeline([('tfidf', TfidfVectorizer(min_df=0.0, analyzer = 'char', 
                                               ngram_range = (1,3), sublinear_tf=True)),
                     ('lgs', LogisticRegression(multi_class='multinomial', class_weight = 'balanced', 
                                                random_state=10))])

grid_f1 = GridSearchCV(pipeline, param_grid=mf_grid, n_jobs=5, cv=skfold, scoring='f1_macro')
grid_f1.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer='char',
                                                        min_df=0.0,
                                                        ngram_range=(1, 3),
                                                        sublinear_tf=True)),
                                       ('lgs',
                                        LogisticRegression(class_weight='balanced',
                                                           multi_class='multinomial',
                                                           random_state=10))]),
             n_jobs=5,
             param_grid={'tfidf__max_features': [15000, 100000, 110000, 120000,
                                                 130000, 140000, 150000]},
             scoring='f1_macro')

In [70]:
# skfold n_split = 10 일때, 'tfidf__max_features': 140000 best
f1_score_df = pd.DataFrame(grid_f1.cv_results_)
f1_score_df[['params', 'rank_test_score', 'mean_test_score', 'std_test_score']]

,params,rank_test_score,mean_test_score,std_test_score
0,{'tfidf__max_features': 15000},7,0.562470,0.019643
1,{'tfidf__max_features': 100000},3,0.567885,0.015031
2,{'tfidf__max_features': 110000},4,0.567667,0.015615
3,{'tfidf__max_features': 120000},5,0.567046,0.014611
4,{'tfidf__max_features': 130000},6,0.566957,0.015243
5,{'tfidf__max_features': 140000},1,0.568353,0.016462
6,{'tfidf__max_features': 150000},1,0.568353,0.016462


In [51]:
# skfold n_split = 5 일때, 'tfidf__max_features': 120000 best

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tfidf__max_features,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.890541,0.008333,0.104720,0.004678,15000,{'tfidf__max_features': 15000},0.544547,0.560984,0.547734,0.583472,0.569803,0.561308,0.014343,7
1,9.760679,0.413627,0.170145,0.017948,100000,{'tfidf__max_features': 100000},0.551001,0.577542,0.557118,0.581477,0.569393,0.567306,0.011659,5
2,10.184744,0.344920,0.152391,0.012156,110000,{'tfidf__max_features': 110000},0.552651,0.576906,0.555061,0.581384,0.568937,0.566988,0.011465,6
3,10.325765,0.763595,0.186301,0.028873,120000,{'tfidf__max_features': 120000},0.555232,0.575109,0.557750,0.580376,0.569113,0.567516,0.009714,1
4,12.660319,0.533350,0.157976,0.008570,130000,{'tfidf__max_features': 130000},0.553542,0.576121,0.558690,0.580913,0.567948,0.567443,0.010265,2
5,12.674771,0.357641,0.185303,0.032060,140000,{'tfidf__max_features': 140000},0.553542,0.576121,0.558690,0.580913,0.567948,0.567443,0.010265,2
6,12.945357,0.455758,0.145210,0.029294,150000,{'tfidf__max_features': 150000},0.553542,0.576121,0.558690,0.580913,0.567948,0.567443,0.010265,2


### 3. dev로 평가
- skfold = 5 일때, max_features = 120000 가장좋음
- skfold = 10 일때, max_features = 140000 가장좋음
- f1 score로 따로 확인해 봤을때는 140000 이 더 좋음
- 결과에 의해서는 10개로 쪼갰을때가 조금더 정확함

In [41]:
dev_df = pd.read_csv('./korean-hate-speech-master/labeled/dev.tsv', sep='\t')

dev_df.head()
dev_df = dev_df[['comments', 'hate']]
dev_df.columns = ['comments', 'label']
dev_df.tail()

,comments,label
466,지현우 범죄 저지르지 않았나요?,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate
470,남자가 잘못한거라면... 반성도 없다면...나였다면 ... 여자처럼 아주 못되게 할...,none


In [69]:
vec_tf = TfidfVectorizer(min_df = 0.0, analyzer = 'char', sublinear_tf=True, 
    ngram_range = (1,3), max_features=140000)

vec_tf.fit(X)
X_vec = vec_tf.transform(X)
X_tf = vec_tf.transform(dev_df['comments'])
y_tf = dev_df['label']

In [59]:
print(X_tf.shape)
X_tf.toarray()

(471, 100000)


array([[0.05583205, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0401012 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03900551, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.0590684 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.050873  , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04562361, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [71]:
lgs = LogisticRegression(multi_class='multinomial', class_weight = 'balanced', 
                                                random_state=10)
lgs.fit(X_vec, y)
pred = lgs.predict(X_tf)

In [72]:
from sklearn.metrics import f1_score

f1_score(y_tf, pred, average='macro')

0.6216400857499588

In [61]:
from sklearn.metrics import f1_score

f1_score(y_tf, pred, average='macro')

0.6171504866913602

### 4. test 검증

In [73]:
test_df = pd.read_csv('./korean-hate-speech-master/test.no_label.tsv', sep='\t')

X_test = vec_tf.transform(test_df['comments'])
pred = lgs.predict(X_test)
test_df['label'] = pred

In [74]:
df = kaggle_format(test_df)
df.to_csv('140000.csv', index=False)

## 실험 01-03
- skfold n_split = 10 인 상태로
- max_features = 140000 ~ 150000사이에 최적의 값 확인
- max_features = 140000 / grid : 0.568353 / kaggle : 0.52893

### 1. 데이터 다시 읽기

In [75]:
df = pd.read_csv('total_20210121.csv')
df = df[['title', 'comments', 'hate']]
df.columns = ['title', 'comments', 'label']

X, y = df['comments'], df['label']

### 2. GridSearchCV 최적값 확인
- 1000단위로 확인
- 그다음 100단위로 확인

In [79]:
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [80]:
# 적용
skfold = StratifiedKFold(n_splits=10)

mf_grid = {
    'tfidf__max_features': list(range(140000, 150001, 1000)),
}
pipeline = Pipeline([('tfidf', TfidfVectorizer(min_df=0.0, analyzer = 'char', 
                                               ngram_range = (1,3), sublinear_tf=True)),
                     ('lgs', LogisticRegression(multi_class='multinomial', class_weight = 'balanced', 
                                                random_state=10))])

grid_f1 = GridSearchCV(pipeline, param_grid=mf_grid, n_jobs=5, cv=skfold, scoring='f1_macro')
grid_f1.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(analyzer='char',
                                                        min_df=0.0,
                                                        ngram_range=(1, 3),
                                                        sublinear_tf=True)),
                                       ('lgs',
                                        LogisticRegression(class_weight='balanced',
                                                           multi_class='multinomial',
                                                           random_state=10))]),
             n_jobs=5,
             param_grid={'tfidf__max_features': [140000, 141000, 142000, 143000,
                                                 144000, 145000, 146000, 147000,
                                                 148000, 149000, 150000]},
             scoring

In [81]:
# 확인
f1_score_df = pd.DataFrame(grid_f1.cv_results_)
f1_score_df[['params', 'rank_test_score', 'mean_test_score', 'std_test_score']]

,params,rank_test_score,mean_test_score,std_test_score
0,{'tfidf__max_features': 140000},1,0.568353,0.016462
1,{'tfidf__max_features': 141000},1,0.568353,0.016462
2,{'tfidf__max_features': 142000},1,0.568353,0.016462
3,{'tfidf__max_features': 143000},1,0.568353,0.016462
4,{'tfidf__max_features': 144000},1,0.568353,0.016462
5,{'tfidf__max_features': 145000},1,0.568353,0.016462
6,{'tfidf__max_features': 146000},1,0.568353,0.016462
7,{'tfidf__max_features': 147000},1,0.568353,0.016462
8,{'tfidf__max_features': 148000},1,0.568353,0.016462
9,{'tfidf__max_features': 149000},1,0.568353,0.016462


In [82]:
# 이 결과는 포기하고,, ngram 변화로 확인해야 할듯